# LLM Brochure

## This project
- Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits
- provide a company name and their primary website

## Business Use Cases
First example of Agentic AI design patterns, as we combined multiple calls to LLMs. Generating content in this way is one of the very most common Use Cases.
- Can be applied to any business vertical
    - Summarization
    - Write marketing content
    - Generate product tutorial from a spec
    - Create email content
    - Many, many more 



In [49]:
import os
import requests
import json
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display 
from openai import OpenAI

In [44]:
#initializing constants
llama_url = 'http://localhost:11434/v1'
llama_api = "http://localhost:11434/api/chat"
Headers = {"Content-Type": "application/json"}
Model = "llama3.1"

Anthropic_url = "https://www.anthropic.com/"

In [ ]:
# Utility Class - Expand on website class from ollam website summary project
class Website:
    def __init__(self, url):
        self.url = url

        response = requests.get(url)
        #return content from url 
        self.body = response.content
        soup = BeautifulSoup(response.content, 'html.parser')

        self.title = soup.title.string if soup.title else "No Title"

        if soup.body:
            # remove unneeded objects from scrape for summarization
            for irrelevant_objects in soup.body(["script", "style", "img", "input"]):
                irrelevant_objects.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link] #Build a new list with only the truthy (non-empty, non-None, non-zero) items from links

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\n Webpage Contents:\n{self.text}\n\n"

In [24]:
anthropic_website = Website("https://www.anthropic.com/")
anthropic_website.links

['#main',
 '#footer',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/claude-code',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://www.anthropic.com/pricing',
 'https://claude.ai/download',
 'https://claude.ai/',
 'https://www.anthropic.com/news/claude-character',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/pricing#api',
 'https://console.anthropic.com/',
 'https://docs.anthropic.com/en/docs/welcome',
 'https://www.anthropic.com/solutions/agents',
 'https://www.anthropic.com/solutions/code-modernization',
 'https://www.anthropic.com/solutions/coding',
 'https://www.anthropic.com/solutions/customer-support',
 'https://www.anthropic.com/solutions/education',
 'https://www.anthropic.com/solutions/financial-services',
 'https://www.anthropic.com/solutions/government',
 'https://www.anthropic.com/customers',
 'https://www.

### Links System Prompts

Use a call to llama to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. 

In [18]:
# System Prompts
links_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
links_system_prompt += "You should respond in JSON as in this example:"
links_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [19]:
# Create User Prompt
def get_links_for_user_prompt(website):
    user_prompt = f"Here are a list of links for {website.url}"
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
    Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [23]:
print(get_links_for_user_prompt(anthropic_website))

Here are a list of links for https://www.anthropic.com/please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format.     Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#main
#footer
https://www.anthropic.com/
https://www.anthropic.com/claude
https://www.anthropic.com/claude-code
https://www.anthropic.com/max
https://www.anthropic.com/team
https://www.anthropic.com/enterprise
https://www.anthropic.com/pricing
https://claude.ai/download
https://claude.ai/
https://www.anthropic.com/news/claude-character
https://www.anthropic.com/api
https://docs.anthropic.com/
https://www.anthropic.com/pricing#api
https://console.anthropic.com/
https://docs.anthropic.com/en/docs/welcome
https://www.anthropic.com/solutions/agents
https://www.anthropic.com/solutions/code-modernization
https://www.anthropic.com/solutions/coding
https://www.anthropic.com/solutions/customer-support
https://www.

In [30]:
#function that puts everything together and gets links - remember I can use OpenAI to call llama3.1
def get_links(url):
    website = Website(url)
    #calling local llama3.1 model
    ollama_via_openai = OpenAI(base_url=llama_url, api_key="ollama")

    response = ollama_via_openai.chat.completions.create(
        model= Model,
        messages=[
            {"role":"system", "content":links_system_prompt},
            {"role":"user", "content":get_links_for_user_prompt(website)}
            ],
        response_format= {"type":"json_object"}
    )

    results = response.choices[0].message.content
    return json.loads(results)



In [33]:
#get all links for anthropic brochures
get_links("https://www.anthropic.com/")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'partners page', 'url': 'https://www.anthropic.com/partners/mcp'},
  {'type': 'partners page',
   'url': 'https://www.anthropic.com/partners/powered-by-claude'},
  {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'},
  {'type': 'solutions page',
   'url': 'https://www.anthropic.com/solutions/agents'},
  {'type': 'solutions page',
   'url': 'https://www.anthropic.com/solutions/code-modernization'},
  {'type': 'solutions page',
   'url': 'https://www.anthropic.com/solutions/coding'},
  {'type': 'solutions page',
   'url': 'https://www.anthropic.com/solutions/customer-support'},
  {'type': 'solutions page',
   'url': 'https://www.anthropic.com/solutions/education'},
  {'type': 'solutions page',
   'url': 'https://www.anthropic.com/solutions/financial-services'},
  {'type': 'solutions page',
   'url': 'https:/

## Brochure Creation
Putting together the brochure by:
1. gathering details
2. creating brochure prompt

In [37]:
def gather_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result 

In [39]:
print(gather_details("https://www.anthropic.com/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions/agents'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions/customer-support'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'careers web page (jobs) ', 'url': 'https://www.anthropic.com/jobs'}]}
Landing page:
Webpage Title:
Home \ Anthropic
 Webpage Contents:
Skip to main content
Skip to footer
Claude
Product
Claude
Claude Code
Plans
Max plan
Team plan
Enterprise plan
Explore pricing
Download apps
Claude log in
News
Claude’s Character
API
Build with Claude
API overview
Developer docs
Explore pricing
Console log in
News
Learn how to build with Claude
Solutions
Collabora

In [41]:
#create system prompt
brochure_system_prompt = "You are an marketing assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone'
# brochure_system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [42]:
def get_user_prompt_for_brochure(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += gather_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [46]:
print(get_user_prompt_for_brochure("Anthropic", Anthropic_url))

Found links: {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'solutions page (agents)', 'url': 'https://www.anthropic.com/solutions/agents'}, {'type': 'solutions page (code modernization)', 'url': 'https://www.anthropic.com/solutions/code-modernization'}, {'type': 'solutions page (coding)', 'url': 'https://www.anthropic.com/solutions/coding'}, {'type': 'solutions page (customer support)', 'url': 'https://www.anthropic.com/solutions/customer-support'}, {'type': 'solutions page (education)', 'url': 'https://www.anthropic.com/solutions/education'}, {'type': 'solutions page (financial services)', 'url': 'https://www.anthropic.com/solutions/financial-services'}, {'type':

In [ ]:
def create_brochure(company_name, url):
    ollama_via_openai = OpenAI(base_url=llama_url, api_key='ollama')
    response = ollama_via_openai.chat.completions.create(
        model=Model,
        messages=[
            {"role":"system", "content": brochure_system_prompt}, 
            {"role":"user", "content": get_user_prompt_for_brochure(company_name,url)}
        ]
    )
    result = response.choices[0].message.content

    display(Markdown(result))

In [51]:
#Put all together and create Brochure
create_brochure("Anthropic", Anthropic_url)

Found links: {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'company page', 'url': 'https://www.anthropic.com/team'}, {'type': 'about company', 'url': 'https://www.anthropic.com/claude'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}, {'type': 'learn more page', 'url': 'https://www.anthropic.com/learn'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'company news page', 'url': 'https://www.anthropic.com/news'}]}


# Welcome to Anthropic

## Company Overview
At Anthropic, we believe that artificial intelligence can be a powerful tool for serving humanity's long-term well-being. Our mission is to build AI tools with human benefit at their foundation.

### About Us
We focus on designing and developing technologies that prioritize responsibility and safety in the face of uncertainty. We aim to show what responsible AI development looks like in practice through our daily research, policy work, and product design.

## Product - Claude

Claude is a highly advanced language model developed by Anthropic. It's designed to assist with coding tasks and can handle hours of work across multiple projects at once. Our models, including Claude Opus 4.1 and Sonnet 4, push the frontier for AI agents and coding capabilities.

### Research Focus
Our team conducts ongoing research into both the development and governance of large language models. This includes exploring the societal impacts of these technologies and developing methods to ensure safety as they become more powerful.

## Our Approach to AI Safety

We recognize that we cannot foresee every outcome our technologies will have on society, but we are committed to building tools with human benefit at their foundation. We believe this requires both bold steps forward and intentional pauses to consider the effects of these new technologies.

### Core Views on AI Safety

Our views on AI safety are centered on a few key principles:

- **Human-centered design**: We prioritize developing technologies that serve humanity's long-term well-being.
- **Intentional pauses**: Regular evaluations and reflections on the potential impacts of our advancements.
- **Bold progress**: Encouraging innovation and progress in AI while adhering to safety standards.

## Join Our Community

### Careers
If you share our passion for responsible AI development, consider joining our team. We have several open roles available. Speak with our sales team today or explore our careers page to learn more about how you can contribute to shaping the future of safe AI.

### How You Can Help

Want to help us build the future of safe AI? If you're a developer interested in exploring opportunities for integrating Claude into your work, visit our Developer Docs. Or if you're looking for ways to stay updated on our latest research and developments, consider following us through this website or subscribing to any number of our publications.

Note: The structure provided above adheres strictly to README standards but note that not all details might have been included as we are constrained by the information available.

In [52]:
# To make the streaming/type-writter effect like in ChatGPT

def create_brochure_with_stream(company_name, url):
    ollama_via_openai = OpenAI(base_url=llama_url, api_key='ollama')
    #change response from above function to stream
    stream = ollama_via_openai.chat.completions.create(
        model=Model,
        messages=[
            {"role":"system", "content": brochure_system_prompt},
            {"role":"user", "content": get_user_prompt_for_brochure(company_name,url)}
        ],
        #Set stream=True for typewritter effect
        stream=True
    )

    # Initialize an empty string to accumulate the streamed text
    response = ""

    # Create an empty Markdown display in Jupyter that can update later
    display_handle = display(Markdown(""), display_id=True) # 'display_id=True' ensures we can overwrite this output instead of printing new lines

    # Loop through each chunk of the streamed response
    for chunk in stream:
        # Append the new text (if any) from the current chunk to the accumulated response
        response += chunk.choices[0].delta.content or ''

        # Clean up unwanted formatting: remove code fences (```), remove the word 'markdown'
        response = response.replace("```", "").replace("markdown", "")

        # Update the existing Markdown display with the new accumulated text
        # 'display_id' ensures the original display is replaced, giving a live updating effect
        update_display(Markdown(response), display_id=display_handle.display_id)


In [53]:
create_brochure_with_stream("Anthropic", Anthropic_url)


Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'company news/press', 'url': 'https://www.anthropic.com/news'}, {'type': 'customers/success stories', 'url': 'https://www.anthropic.com/customers'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}, {'type': 'learn/education', 'url': 'https://www.anthropic.com/learn'}, {'type': 'research/papers', 'url': 'https://www.anthropic.com/research'}, {'type': 'partners page', 'url': 'https://www.anthropic.com/partners'}, {'type': 'jobs page', 'url': 'https://www.anthropic.com/jobs'}, {'type': 'contact sales', 'url': 'https://www.anthropic.com/contact-sales'}]}


**Anthropic: Building Safe AI for Humanity's Long-term Well-being**

[Header Image: A stylized representation of a human and an artificial intelligence system, with a balance scale in the middle]

At Anthropic, we're dedicated to developing AI that serves humanity's best interests. Our mission is to build tools that align with human values and benefit society as a whole.

**Our Technology: Claude**

Claude is our flagship product, a state-of-the-art large language model designed to assist with tasks such as coding, creative writing, and customer support. With features like natural language understanding, Claude can handle complex queries and provide personalized solutions.

**Key Features of Claude:**

*   **Code Modernization**: Use Claude to modernize your codebase and improve performance.
*   **Coding Assistance**: Get help with coding tasks, from debugging to implementation.
*   **API Integration**: Easily integrate Claude into your applications using our API platform.
*   **Enterprise-ready**: Scale your operations with Claude's flexible pricing plans.

**How We Approach AI Safety**

To ensure our technology is both powerful and safe, we prioritize:
    *   Core Views on AI Safety: [link to learn more](#)
    *   Responsible Scaling Policy: [link to learn more](#)
    *   Transparency: Regular updates on our progress, challenges, and achievements.

**Our Values**

At Anthropic, we're committed to:

*   **Human benefit as the foundation**: Designing tools that prioritize human well-being.
*   **Responsible innovation**: Balancing bold steps forward with intentional pauses to consider effects.
*   The future of safe AI is a team effort. We invite you to join us in shaping this future by:

    *   [Learn more about our open roles](#)
    *   [Get in touch with our sales team](#)

**Staying Up-to-Date**

Stay informed about the latest developments in our research and products through our:

*   [News archive](#)
*   [Blog posts on AI Safety and responsible innovation](#)

**Join Our Community**

Be part of the conversation shaping the future of safe AI. Engage with us on our social media channels, ask questions, share your thoughts, and collaborate with our team to build a better tomorrow.

---

Anthropic: Safely empowering the next generation of AI technology.